In [23]:
from IPython.display import display
import numpy as np
import pandas as pd
import folium

In [24]:
data_revenus = pd.read_excel('../../data/revenus_communes_2019.xlsx')
data_bars = pd.read_csv('../../data/osm-fr-bars.csv', sep=';')

display(data_revenus, data_bars)

,Unnamed: 0,DGFiP - Bureau des études statistiques en matière fiscale,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IRCOM revenus 2019
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Montants en milliers d'euros (exceptés ceux de...
2,NaN,Dép.,Commune,Libellé de la commune,Revenu fiscal de référence par tranche (en euros),Nombre de foyers fiscaux,Revenu fiscal de référence des foyers fiscaux,Impôt net (total)*,Nombre de foyers fiscaux imposés,Revenu fiscal de référence des foyers fiscaux ...,Traitements et salaires,NaN,Retraites et pensions,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nombre de foyers concernés,Montant,Nombre de foyers concernés,Montant
4,NaN,010,001,L'Abergement-Clémenciat,TOTAL,447,13521.06,738.597,249,9873.482,312,9535.029,161,4017.749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82930,NaN,B31,999,Autres,50 001 à 100 000,443,31302.071,7757.596,n.c.,n.c.,n.c.,n.c.,n.c.,n.c.
82931,NaN,B31,999,Autres,Plus de 100 000,586,108462.346,31927.764,n.c.,n.c.,n.c.,n.c.,n.c.,n.c.
82932,NaN,B31,999,Autres,TOTAL,3963,171885.111,45862.369,2589,170115.095,162,3568.91,31,560.458
82933,NaN,"* Impôt sur le revenu émis par voie de rôle, h...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Geo Point,Nom,Boissons,Site web,Téléphone,Horaires,Accessible PMR,PMR (description),Toilettes PMR,Id Wikidata,Id Wikipédia,Code postal,other_tags
0,"44.0549704,0.2603921",Au boin coin,NaN,NaN,+33 5 53 65 67 45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""cuisine"": ""tapas"", ""ref:FR:CRTA"": ""RESAQU047..."
1,"47.9094426,1.9076031",Le Diabolo Menthe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""source"": ""survey""}"
2,"48.8020856,2.427514",Café de la Gare,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"41.6976671,9.0644945",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"43.788717,4.8299152",Brasserie du Commerce,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13210,"{""addr:housenumber"": ""22"", ""addr:city"": ""Saint..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17928,"44.2761336,5.2758837",Les Glycines,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17929,"46.7227908,2.5037666",La Rotonde,NaN,NaN,+33 2 48961610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17930,"43.8339541,4.3534613",La Movida,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17931,"46.7776345,4.8488139",Nicéphore Café,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
data_bars = data_bars[["Geo Point", "Code postal", "Nom"]]


display(data_bars)
print("Rate of undefined values:", data_bars["Code postal"].isna().sum() / len(data_bars["Code postal"]))

,Geo Point,Code postal,Nom
0,"44.0549704,0.2603921",NaN,Au boin coin
1,"47.9094426,1.9076031",NaN,Le Diabolo Menthe
2,"48.8020856,2.427514",NaN,Café de la Gare
3,"41.6976671,9.0644945",NaN,NaN
4,"43.788717,4.8299152",13210,Brasserie du Commerce
...,...,...,...
17928,"44.2761336,5.2758837",NaN,Les Glycines
17929,"46.7227908,2.5037666",NaN,La Rotonde
17930,"43.8339541,4.3534613",NaN,La Movida
17931,"46.7776345,4.8488139",NaN,Nicéphore Café


Rate of undefined values: 0.8377850889421736


Since the rate of undefined value is way higher than 0.5, and that GPS coordinates and postal code are a little redundant, we can reasonably delete Code postal column:

In [26]:
data_bars.drop("Code postal", axis=1,  inplace=True)
data_bars

,Geo Point,Nom
0,"44.0549704,0.2603921",Au boin coin
1,"47.9094426,1.9076031",Le Diabolo Menthe
2,"48.8020856,2.427514",Café de la Gare
3,"41.6976671,9.0644945",NaN
4,"43.788717,4.8299152",Brasserie du Commerce
...,...,...
17928,"44.2761336,5.2758837",Les Glycines
17929,"46.7227908,2.5037666",La Rotonde
17930,"43.8339541,4.3534613",La Movida
17931,"46.7776345,4.8488139",Nicéphore Café


In [27]:
latitudes = []
longitudes = []

for row in data_bars['Geo Point']:
    try:
        latitudes.append(row.split(',')[0])
        longitudes.append(row.split(',')[1])
    except:
        latitudes.append(np.NaN)
        longitudes.append(np.NaN)

data_bars['Latitude'] = latitudes
data_bars['Longitude'] = longitudes
data_bars.drop('Geo Point', 1)

/var/folders/97/y6ldl6694d978551pjfjtq8m0000gn/T/ipykernel_10438/1849365207.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data_bars.drop('Geo Point', 1)


,Nom,Latitude,Longitude
0,Au boin coin,44.0549704,0.2603921
1,Le Diabolo Menthe,47.9094426,1.9076031
2,Café de la Gare,48.8020856,2.427514
3,NaN,41.6976671,9.0644945
4,Brasserie du Commerce,43.788717,4.8299152
...,...,...,...
17928,Les Glycines,44.2761336,5.2758837
17929,La Rotonde,46.7227908,2.5037666
17930,La Movida,43.8339541,4.3534613
17931,Nicéphore Café,46.7776345,4.8488139


Number of missing values

In [28]:
data_bars.isnull().sum().sum()
data_bars = data_bars.head(100)
data_bars

,Geo Point,Nom,Latitude,Longitude
0,"44.0549704,0.2603921",Au boin coin,44.0549704,0.2603921
1,"47.9094426,1.9076031",Le Diabolo Menthe,47.9094426,1.9076031
2,"48.8020856,2.427514",Café de la Gare,48.8020856,2.427514
3,"41.6976671,9.0644945",NaN,41.6976671,9.0644945
4,"43.788717,4.8299152",Brasserie du Commerce,43.788717,4.8299152
...,...,...,...,...
95,"43.3081618,5.381486",salon de thé Sidi Bou,43.3081618,5.381486
96,"48.8375341,2.4005448",Le Royal Daumesnil,48.8375341,2.4005448
97,"50.6361565,3.0693038",Les 3 Brasseurs,50.6361565,3.0693038
98,"46.3253061,-0.4630548",Place des Halles,46.3253061,-0.4630548


Let's just see that on a map

In [29]:
postalCodes = pd.read_csv('../../data/postal-code.csv')
postalCodes

/var/folders/97/y6ldl6694d978551pjfjtq8m0000gn/T/ipykernel_10438/1003043654.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  postalCodes = pd.read_csv('../../data/postal-code.csv')


,010010005Y-103,103,Impasse des Acacias,01400,L'Abergement-Clémenciat,C+O,46.147624,4.923727
0,010010005Y-104,104,Impasse des Acacias,01400,L'Abergement-Clémenciat,C+O,46.147812,4.924310
1,010010005Y-26,26,Impasse des Acacias,01400,L'Abergement-Clémenciat,C+O,46.146756,4.924307
2,010010005Y-30,30,Impasse des Acacias,01400,L'Abergement-Clémenciat,C+O,46.146907,4.924547
3,010010005Y-59,59,Impasse des Acacias,01400,L'Abergement-Clémenciat,C+O,46.147154,4.924191
4,010010005Y-64,64,Impasse des Acacias,01400,L'Abergement-Clémenciat,C+O,46.147307,4.924624
...,...,...,...,...,...,...,...,...
22845076,97617B740A-4504,4504,Village de Tsingoni,97680,Tsingoni,CAD,-12.790557,45.137112
22845077,97617B740A-4506,4506,Village de Tsingoni,97680,Tsingoni,CAD,-12.789817,45.132364
22845078,97617B740A-4524,4524,Village de Tsingoni,97680,Tsingoni,CAD,-12.788548,45.132566
22845079,97617B740A-5000,5000,Village de Tsingoni,97680,Tsingoni,CAD,-12.789423,45.103516


In [30]:
postalCodes.drop(postalCodes.columns[[0,1,2,4,5]], 1, inplace=True)
postalCodes.columns = ['Postal code', 'Latitude', 'Longitude']
postalCodes

/var/folders/97/y6ldl6694d978551pjfjtq8m0000gn/T/ipykernel_10438/3129582374.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  postalCodes.drop(postalCodes.columns[[0,1,2,4,5]], 1, inplace=True)


,Postal code,Latitude,Longitude
0,01400,46.147812,4.924310
1,01400,46.146756,4.924307
2,01400,46.146907,4.924547
3,01400,46.147154,4.924191
4,01400,46.147307,4.924624
...,...,...,...
22845076,97680,-12.790557,45.137112
22845077,97680,-12.789817,45.132364
22845078,97680,-12.788548,45.132566
22845079,97680,-12.789423,45.103516


Le meilleur compromis d'arrondi des coordonnées GPS est de 3 car pour un arrondi de 4 nous obtenons 16749 valeurs non trouvées et en dessous de 3 la précision est trop faible : on peut obtenir un code postal erroné. Nous aurions pu faire un appel à une API pour avoir des valeurs exacte facilement, mais nous trouvions que c'était moins dans l'esprit du projet.
9859; 1524; 617;

In [31]:
postalCodes['Latitude'] = postalCodes['Latitude'].apply(lambda lat: round(lat, 3))
postalCodes['Longitude'] = postalCodes['Longitude'].apply(lambda long: round(long, 3))

data_bars['Latitude'] = data_bars['Latitude'].astype(float)
data_bars['Longitude'] = data_bars['Longitude'].astype(float)
data_bars['Latitude'] = data_bars['Latitude'].apply(lambda lat: round(lat, 3))
data_bars['Longitude'] = data_bars['Longitude'].apply(lambda long: round(long, 3))

data_bars = pd.merge(data_bars, postalCodes, how='left', on=['Latitude', 'Longitude']).drop_duplicates()
data_bars

/var/folders/97/y6ldl6694d978551pjfjtq8m0000gn/T/ipykernel_10438/1994631245.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bars['Latitude'] = data_bars['Latitude'].astype(float)
/var/folders/97/y6ldl6694d978551pjfjtq8m0000gn/T/ipykernel_10438/1994631245.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bars['Longitude'] = data_bars['Longitude'].astype(float)
/var/folders/97/y6ldl6694d978551pjfjtq8m0000gn/T/ipykernel_10438/1994631245.py:6: SettingWithCopyWarning: 
A value is trying to be set

,Geo Point,Nom,Latitude,Longitude,Postal code
0,"44.0549704,0.2603921",Au boin coin,44.055,0.260,47170
45,"47.9094426,1.9076031",Le Diabolo Menthe,47.909,1.908,45000
48,"48.8020856,2.427514",Café de la Gare,48.802,2.428,94700
62,"41.6976671,9.0644945",NaN,41.698,9.064,NaN
63,"43.788717,4.8299152",Brasserie du Commerce,43.789,4.830,13210
...,...,...,...,...,...
2003,"43.3081618,5.381486",salon de thé Sidi Bou,43.308,5.381,13003
2026,"48.8375341,2.4005448",Le Royal Daumesnil,48.838,2.401,75012
2039,"50.6361565,3.0693038",Les 3 Brasseurs,50.636,3.069,59800
2077,"46.3253061,-0.4630548",Place des Halles,46.325,-0.463,79000


In [32]:
data_bars.dropna(subset=['Postal code'], inplace=True)
data_bars['Département'] = data_bars['Postal code'].apply(lambda code: str(code)[0:2])
data_bars

,Geo Point,Nom,Latitude,Longitude,Postal code
0,"44.0549704,0.2603921",Au boin coin,44.055,0.260,47170
45,"47.9094426,1.9076031",Le Diabolo Menthe,47.909,1.908,45000
48,"48.8020856,2.427514",Café de la Gare,48.802,2.428,94700
63,"43.788717,4.8299152",Brasserie du Commerce,43.789,4.830,13210
101,"45.5853255,5.2768205",L'atelier de Chico,45.585,5.277,38300
...,...,...,...,...,...
2003,"43.3081618,5.381486",salon de thé Sidi Bou,43.308,5.381,13003
2026,"48.8375341,2.4005448",Le Royal Daumesnil,48.838,2.401,75012
2039,"50.6361565,3.0693038",Les 3 Brasseurs,50.636,3.069,59800
2077,"46.3253061,-0.4630548",Place des Halles,46.325,-0.463,79000


In [13]:
barNumber = data_bars['Département'].groupby('Département', as_index=False).sum()

,Geo Point,Nom,Latitude,Longitude,Postal code
0,"44.0549704,0.2603921",Au boin coin,44.055,0.260,47170
45,"47.9094426,1.9076031",Le Diabolo Menthe,47.909,1.908,45000
48,"48.8020856,2.427514",Café de la Gare,48.802,2.428,94700
62,"41.6976671,9.0644945",NaN,41.698,9.064,NaN
63,"43.788717,4.8299152",Brasserie du Commerce,43.789,4.830,13210
...,...,...,...,...,...
385913,"44.2761336,5.2758837",Les Glycines,44.276,5.276,26170
385945,"46.7227908,2.5037666",La Rotonde,46.723,2.504,18200
385960,"43.8339541,4.3534613",La Movida,43.834,4.353,30900
386008,"46.7776345,4.8488139",Nicéphore Café,46.778,4.849,71100


In [ ]:
import plotly.express as px
import json

departements = json.load(open('data/departements-version-simplifiee.geojson'))

fig = px.choropleth_mapbox(barNumber, geojson=departements,
                           locations='département', featureidkey = 'properties.code', # join keys
                           color='prix', color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           zoom=4.6, center = {"lat": 47, "lon": 2},
                           opacity=0.5,
                           labels={'prix':'Nombre de bars'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig

In [52]:
# for lat,lon,name in zip(data_bars['Latitude'], data_bars['Longitude'], data_bars['Nom']):
#     folium.Marker((lat,lon), popup=name).add_to(france)
# vb = data_bars.iloc[0]
# france

# france = folium.Map(location=[46.5,2], zoom_start=5, tiles='Stamen Toner')
#
# colors = folium.Choropleth(geo_data = 'data/departements-version-simplifiee.geojson',
#                            data = barNumber,
#                            columns = ['département', 'prix'],
#                            key_on = 'feature.properties.code',
#                            fill_color = 'YlOrRd',
#                            fill_opacity = 0.6,
#                            line_opacity = 0.2,
#                            legend_name = "Prix de l\\'essence E10"
#                           )
# colors.add_to(france)
# folium.LayerControl().add_to(france)
# france

In [ ]:
data_revenus.rename(columns=lambda n : data_revenus[n][2], inplace= True)
data_revenus.drop(index=[0, 1, 2], inplace=True)
data_revenus

In [ ]:
data_revenus["Revenu fiscal de référence par tranche (en euros)"] = data_revenus["Revenu fiscal de référence par tranche (en euros)"].str.strip()
data_revenus = data_revenus[data_revenus["Revenu fiscal de référence par tranche (en euros)"] == 'TOTAL']
data_revenus